Most burden disorder in Poland and its neighboors

In [ ]:
import pandas as pd
from countryinfo import CountryInfo
import pycountry

data = pd.read_csv("./../data/source/2- burden-disease-from-each-mental-illness(1).csv")
file = pd.DataFrame(columns=["Country", "Ranking", "Disorder", "Percentage top50"])

# Getting neighboors for poland
country = CountryInfo("Poland")
neighboors = country.borders()
neighboors.append("POL")

data_countries = data[data["Code"].isin(neighboors)]
data_countries.columns = [
    col.split(" Cause: ")[1].strip() if " Cause:" in col else col
    for col in data_countries.columns
]
data_countries = data_countries.drop(data_countries.columns[1], axis=1)

countries = data_countries["Entity"].unique().tolist()

# Higest burden of disorder
for country in countries:
    data_country = data_countries[data_countries["Entity"] == country]
    data_country = pd.melt(
        data_country,
        id_vars=["Entity", "Year"],
        var_name="Disorder",
        value_name="Daily Rate",
    )

    data_country = data_country.sort_values("Daily Rate", ascending=False)

    # Making up new file with comparison
    top = data_country.head(50)
    disorders = top["Disorder"].unique()

    for disorder in disorders:
        number = top[top["Disorder"] == disorder].shape[0]
        percentage = number / 30
        percentage = percentage * 100
        percentage = round(percentage, 2)
        new_record = [
            {
                "Country": country,
                "Ranking": "Top",
                "Disorder": disorder,
                "Percentage top50": percentage,
            }
        ]
        file = pd.concat([file, pd.DataFrame(new_record)], ignore_index=True)

# Lowest burden of disorder

for country in countries:
    data_country = data_countries[data_countries["Entity"] == country]
    data_country = pd.melt(
        data_country,
        id_vars=["Entity", "Year"],
        var_name="Disorder",
        value_name="Daily Rate",
    )
    data_country = data_country.sort_values("Daily Rate", ascending=True)
    top = data_country.head(50)
    disorders = top["Disorder"].unique()

    for disorder in disorders:
        number = top[top["Disorder"] == disorder].shape[0]
        percentage = number / 30
        percentage = percentage * 100
        percentage = round(percentage, 2)
        # print(disorder+" makes up ", percentage,"% of coverage with lowest 50 disorder")

        new_record = [
            {
                "Country": country,
                "Ranking": "Low",
                "Disorder": disorder,
                "Percentage top50": percentage,
            }
        ]
        file = pd.concat([file, pd.DataFrame(new_record)], ignore_index=True)

file.to_csv("./../results/exstremes_burder_neighboors.csv", index=False, header=True)

Heat map for disorders, countries in dependence of time for each disorder

In [ ]:
# Makes files for each disorder
column_list = data_countries.columns
print(column_list)
disorders_list = column_list[2:]

for disorder in disorders_list:
    print(disorder)
    file_disorder = pd.DataFrame(columns=["Country", "Year", "DALYs"])
    file_disorder = data_countries[["Entity", "Year", disorder]]
    file_disorder = file_disorder.rename(
        columns={"Entity": "Country", disorder: "DALYs"}
    )
    print(file_disorder)
    file_disorder.to_csv(
        "./../data/burden/neighboors_" + disorder, header=True, index=False
    )